<a href="https://colab.research.google.com/github/microsoft/autogen/blob/main/notebook/agentchat_web_info.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Auto Generated Agent Chat: Solving Tasks Requiring Web Info

AutoGen offers conversable agents powered by LLM, tool, or human, which can be used to perform tasks collectively via automated chat. This framework allows tool use and human participation through multi-agent conversation.
Please find documentation about this feature [here](https://microsoft.github.io/autogen/docs/Use-Cases/agent_chat).

In this notebook, we demonstrate how to use `AssistantAgent` and `UserProxyAgent` to perform tasks which require acquiring info from the web:
* discuss a paper based on its URL.
* discuss about the stock market.

Here `AssistantAgent` is an LLM-based agent that can write Python code (in a Python coding block) for a user to execute for a given task. `UserProxyAgent` is an agent which serves as a proxy for a user to execute the code written by `AssistantAgent`. By setting `human_input_mode` properly, the `UserProxyAgent` can also prompt the user for feedback to `AssistantAgent`. For example, when `human_input_mode` is set to "TERMINATE", the `UserProxyAgent` will execute the code written by `AssistantAgent` directly and return the execution results (success or failure and corresponding outputs) to `AssistantAgent`, and prompt the user for feedback when the task is finished. When user feedback is provided, the `UserProxyAgent` will directly pass the feedback to `AssistantAgent`.

## Requirements

AutoGen requires `Python>=3.8`. To run this notebook example, please install pyautogen and docker:
```bash
pip install pyautogen docker
```

In [1]:
# %pip install pyautogen~=0.1.0 docker

## Set your API Endpoint

The [`config_list_from_json`](https://microsoft.github.io/autogen/docs/reference/oai/openai_utils#config_list_from_json) function loads a list of configurations from an environment variable or a json file.


In [1]:
import autogen

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        # "model": ["gpt4", "gpt-4-32k", "gpt-4-32k-0314", "gpt-4-32k-v0314"],
        "model": ["gpt-35-turbo-16k-dep-001"]
    },
)

llm_config={
    "request_timeout": 600,
    "seed": 42,
    "config_list": config_list,
    "temperature": 0,
}

/home/awesomeyuer/miniconda3/envs/autogen-py311-env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


It first looks for environment variable "OAI_CONFIG_LIST" which needs to be a valid json string. If that variable is not found, it then looks for a json file named "OAI_CONFIG_LIST". It filters the configs by models (you can filter by other keys as well). Only the models with matching names are kept in the list based on the filter condition.

The config list looks like the following:
```python
config_list = [
    {
        'model': 'gpt-4-32k',
        'api_key': '<your OpenAI API key here>',
    },
    {
        'model': 'gpt4',
        'api_key': '<your Azure OpenAI API key here>',
        'api_base': '<your Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2023-06-01-preview',
    },
    {
        'model': 'gpt-4-32k-0314',
        'api_key': '<your Azure OpenAI API key here>',
        'api_base': '<your Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2023-06-01-preview',
    },
]
```

If you open this notebook in colab, you can upload your files by clicking the file icon on the left panel and then choosing "upload file" icon.

You can set the value of config_list in other ways you prefer, e.g., loading from a YAML file.

## Construct Agents

We construct the assistant agent and the user proxy agent. We specify `human_input_mode` as "TERMINATE" in the user proxy agent, which will ask for human feedback when it receives a "TERMINATE" signal from the assistant agent.

In [2]:
# create an AssistantAgent instance named "assistant"
assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config=llm_config,
)
# create a UserProxyAgent instance named "user_proxy"
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    # human_input_mode="TERMINATE",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={"work_dir": "web"},
    llm_config=llm_config,
    system_message="""Reply TERMINATE if the task has been solved at full satisfaction.
Otherwise, reply CONTINUE, or the reason why the task is not solved yet."""
)

## Example Task: Paper Talk from URL

We invoke the `initiate_chat()` method of the user proxy agent to start the conversation. When you run the cell below, you will be prompted to provide feedback after the assistant agent sends a "TERMINATE" signal at the end of the message. If you don't provide any feedback (by pressing Enter directly), the conversation will finish. Before the "TERMINATE" signal, the user proxy agent will try to execute the code suggested by the assistant agent on behalf of the user.

In [3]:
# the assistant receives a message from the user, which contains the task description
user_proxy.initiate_chat(
    assistant,
#     message="""
# Who should read this paper: https://arxiv.org/abs/2308.08155
# """,
    message="""
谁应该阅读这篇论文: https://arxiv.org/abs/2308.08155
""",
)

user_proxy (to assistant):


谁应该阅读这篇论文: https://arxiv.org/abs/2308.08155


--------------------------------------------------------------------------------
assistant (to user_proxy):

要确定谁应该阅读这篇论文，我们可以查看论文的标题和摘要，以及作者的背景和研究领域。请允许我下载并查看论文的内容。

--------------------------------------------------------------------------------
user_proxy (to assistant):

很抱歉，作为语言模型，我无法下载或查看特定的论文内容。但是，您可以根据论文的标题和摘要来判断谁应该阅读这篇论文。通常情况下，适合阅读这篇论文的人包括与论文主题相关的研究人员、学生、教授以及对该领域感兴趣的人。您可以查看作者的背景和研究领域，以确定他们是否与您的兴趣和专业领域相关。

--------------------------------------------------------------------------------
assistant (to user_proxy):

非常抱歉，由于我无法查看论文的内容，我无法提供更具体的建议。但是，根据论文的标题和摘要，您可以尝试以下步骤来确定谁应该阅读这篇论文：

1. 阅读论文的标题和摘要，了解论文的主题和内容。
2. 查看作者的背景和研究领域，了解他们的专业领域和研究方向。
3. 根据论文的主题和作者的背景，确定是否与您的兴趣和专业领域相关。
4. 如果您是研究人员、学生或教授，并且该论文与您的研究领域相关，那么您可能会对阅读该论文感兴趣。
5. 如果您对该领域感兴趣，但不一定是专业人士，您也可以尝试阅读该论文，以扩展您的知识和理解。

请记住，这只是一般建议，具体情况可能因个人兴趣和背景而异。最好的方法是仔细阅读论文的标题和摘要，并根据自己的需求和兴趣来决定是否阅读该论文。

------------------------------------------------------------------

## Example Task: Chat about Stock Market

In [5]:
user_proxy.initiate_chat(
    assistant,
    message="""Show me the YTD gain of 10 largest technology companies as of today.""",
    # message="""展示今天截至目前为止，10家最大的科技公司的年初至今收益。""",
)

user_proxy (to assistant):

Show me the YTD gain of 10 largest technology companies as of today.

--------------------------------------------------------------------------------
assistant (to user_proxy):

To calculate the Year-to-Date (YTD) gain of the 10 largest technology companies, we need the historical stock prices for each company. We can use the yfinance library in Python to fetch the historical data and calculate the YTD gain.

Here's the Python code to fetch the historical stock prices and calculate the YTD gain for the 10 largest technology companies:

```python
# filename: ytd_gain.py
import yfinance as yf
import pandas as pd

# Define the ticker symbols for the 10 largest technology companies
ticker_symbols = ['AAPL', 'MSFT', 'AMZN', 'GOOGL', 'GOOG', 'FB', 'TSM', 'NVDA', 'PYPL', 'INTC']

# Fetch the historical data for the ticker symbols
data = yf.download(ticker_symbols, start='2021-01-01', end='2021-12-31')

# Calculate the YTD gain for each ticker symbol
ytd_gains = {}